## Calculando Containment


Neste notebook, iremos implementar uma funação containment. Tal função irá comparar dois textose analisar a similaridade dos mesmos com relação aos seus n-gramas de interseção.


## Contar N-gram counts

Primeiramente temos que contar as ocorrências de n-gramas dos nossos textos. Usaremos o CountVectorizer para converter o conjunto de dados de texto em uma matriz de contagem.

No código abaixo, podemos variar o valor de n e utilizar o CountVectorizer para contar as ocorrências de n gramas. Podemos notar que na célula abaixo estamos criando um vocabulário através da utilização do CountVectorizer e, posteriormente, iremos analisar a matriz de contagem.

In [1]:
import numpy as np
import sklearn


### Unigrama


A execução do exemplo imprime o vocabulário. Podemos ver que existem 8 palavras no vocabulário e, portanto, vetores codificados também possuem um comprimento de 8.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

a_text = "Suponha que essa seja o texto principal"
s_text = "Suponha que essa seja o texto que desejo comparar"

# Número de n_gramas
n = 1

# Instancia o contador de n-gramas
counts = CountVectorizer(analyzer='word', ngram_range=(n,n))

# Cria um dicionário de n-gramas 
vocab2int = counts.fit([a_text, s_text]).vocabulary_

# Printa dicionário de palavraś:index
print(vocab2int)

{'suponha': 6, 'que': 4, 'essa': 2, 'seja': 5, 'texto': 7, 'principal': 3, 'desejo': 1, 'comparar': 0}


### Bigrama

O mesmo vale para o caso dec bigramas. Temos 8 bigramas no vocabulário e ,portanto, os vetore codificados com comprimento 8.

In [3]:
# Número de n_gramas
n = 2
    
# Instancia o contador de n-gramas
counts = CountVectorizer(analyzer='word', ngram_range=(n,n))

# Cria um dicionário de n-gramas 
vocab2int = counts.fit([a_text, s_text]).vocabulary_

# Printa dicionário de palavraś:index
print(vocab2int)

{'suponha que': 5, 'que essa': 3, 'essa seja': 1, 'seja texto': 4, 'texto principal': 6, 'texto que': 7, 'que desejo': 2, 'desejo comparar': 0}


### Trigrama

In [4]:
# Número de n_gramas
n = 3

# Instancia o contador de n-gramas
counts = CountVectorizer(analyzer='word', ngram_range=(n,n))

# Cria um dicionário de n-gramas 
vocab2int = counts.fit([a_text, s_text]).vocabulary_

# Printa dicionário de palavraś:index
print(vocab2int)

{'suponha que essa': 5, 'que essa seja': 2, 'essa seja texto': 0, 'seja texto principal': 3, 'seja texto que': 4, 'texto que desejo': 6, 'que desejo comparar': 1}


### As palavras do vocabulário

Note que o artigo "o" das frases a_text e s_text não aparece no vocabulário. Note ainda que todas as frases encontram-se em minúsculo. Isso ocorre devido ao fato de que quando passamos o parâmetro **analyser = 'word'**, estamos considerando em nossa análise palavras com dois ou mais caracteres e consequentemente ignrando as palavras com apenas um caracter. Excluir esses caracteres é um comportamento padrão e desejado em muitas análises de texto devido a sua irrelevancia, em grande parte das análises textuais.

Caso você precise desconsiderar o padrão default do CountVectorizer e adicionar palavras com caracteres únicos em sua análise, você pode adicionar o argumento **token_pattern = r"(?u)\b\w+\b"**. Essa expressão regular (REGEX) define palavra como tendo uma ou mais caracteres.

# Array de n-gramas

Vamos usar o CountVectorizer para criar um array com as contagens de n-gramas. Além disso, vamos criar duas frase que desejamos analizar, e transformar cada texto em um vetor numérico representando a ocorrência de cada palavra.

Notar que cada linha representa um texto e cada coluna ou index reprsenta os termos do vocabulário. Iremos ver isso claramente no mapeamento abaixo.

* texto_1 = "Suponha que essa seja o texto principal"
* texto_2 = "Suponha que essa seja o texto que desejo comparar"

In [29]:
# N-gramas 
n = 1

# Instancia o contador de n-gramas
counts = CountVectorizer(analyzer='word', ngram_range=(n,n))

# cria uma matriz de contagem de n-grama para os dois textos
n_grams = counts.fit_transform([texto_1,texto_2])

# Cria um dicionário de n-gramas 
vocab2int = counts.fit([a_text, s_text]).vocabulary_

n_grams_array = n_grams.toarray()
print('Vetor de n-gramas:\n\n', n_grams_array)
print()
print('Dicionário de n-gramas (unigrama):\n\n', vocab2int)

Vetor de n-gramas:

 [[0 0 1 1 1 1 1 1]
 [1 1 1 0 2 1 1 1]]

Dicionário de n-gramas (unigrama):

 {'suponha': 6, 'que': 4, 'essa': 2, 'seja': 5, 'texto': 7, 'principal': 3, 'desejo': 1, 'comparar': 0}


In [ ]:
texto_1 = "Suponha que essa seja o texto principal"
texto_2 = "Suponha que essa seja o texto que desejo comparar"

Acima temos os vetores que codificam cada texto. Na linha superior temos os n-gramas do `text_1` e na linha inferior temos a codificação do `text_2`. Podemos analisar se os textos possuem n_gramas em comum através de suas colunas. Por exemplo, ambos possuem a palavra texo (índice 7 - ultima coluna coluna). O mesmo vale para os unigramas [essa], [seja] e [suponha]. Notar que o unigrama "que" ocorre duas vezes no segundo texto. 


```
[[0 0 1 1 1 1 1 1]    =   ________  ______ [essa] principal [que] [seja] [suponha] [texto]
 [1 1 1 0 2 1 1 1]]   =   comparar  desejo [essa] _________ [que] [seja] [suponha] [texto]
```

# Valores de containment 

O Containment nada mais é do que uma medida de similaridade entre textos. É basicamente uma normalização da interseção da contagem de n-gramas entre os textos.


Primeiro, precisamos extrair as palavras dos dois documentos de texto para formar um corpus.Em seguida, contamos a interseção de n-gramas (agrupamentos seqüenciais de palavras de n palavras) entre os textos 1 e o texto 2. Para o caso de unigramas, podemos considecrar como uma contagem  dos número de palavras que ambos os textos têm em comum.

Em seguida, dividimos o valor pelo total de n-gramas do texto 2 (o qual quer ser comparado com o texto fonte) para normalizar o valor.


Calculo de Containment:

1. Calcular a interseção n-grama entre o texto e o texto fonte.
2. Adicionar o número de termos comuns.
3. Normalizar o valor na etapa 2 pelo número de n gramas no texto 2.


Abaixo podemos ver a equação dec Containment: 
$$ \frac{\sum{count(\text{ngram}_{2}) \cap count(\text{ngram}_{1})}}{\sum{count(\text{ngram}_{2})}} $$


Vamos criar uma função que recebe um array de-gramas

In [40]:
def containment(n_gram_array):
    ''' Calcula o containment entre dois textos. Normaliza a interseção dos contadores de n-gramas
    entre os textos.
    ARG:
    n_gra_array(array): Um array com as contagens de n-gramas dos dois textos a serem comparados
    
    RETURNS:
    O valor de containment normalizado '''
    
    
     # Cria uma lista que contém o valor mínimo encontrado nas colunas
     # 0 se não houver correspondências e 1+ para as palavras correspondentes
    
    intersection_list = np.amin(n_gram_array, axis = 0)
    
    # Soma numero de interseção
    intersection_count = np.sum(intersection_list)
    
    # Conta número dec n-gramas no texto 1
    answer_idx = 0
    answer_count = np.sum(n_gram_array[answer_idx])
    
    
    # Normaliza e calcula valor final 
    containment_val = intersection_count / answer_count
    
    return containment_val  

#### Para o n_gram calculado anteriormente e n = 1

In [41]:

containment_val = containment(n_grams.toarray())

print('Containment: ', containment_val)


Containment:  0.8333333333333334


#### para n = 2

In [42]:
# para n = n
counts_2grams = CountVectorizer(analyzer='word', ngram_range=(2,2))
bigram_counts = counts_2grams.fit_transform([a_text, s_text])

# Calcula containment
containment_val = containment(bigram_counts.toarray())

print('Containment for n=2 : ', containment_val)

Containment for n=2 :  0.8


Teste a função com diferentes textos , n-gramas e tente imaginar aplicações desse conceito. Por exemplo, podemos usar essa técnica como uma métrica de análise de similaridade para detectar plagiarismo.

